<a href="https://colab.research.google.com/github/mohd-faizy/06P_Sentiment_Analysis_With_Deep_Learning_Using_BERT/blob/master/01_Sentiment_Analysis_with_Deep_Learning_using_BERT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# __1. Data Management__

In [ ]:
!pip install torch

In [1]:
import torch
import pandas as pd
from tqdm.notebook import tqdm

In [2]:
df = pd.read_csv('/content/smile-annotations-final.csv', names=['id', 'text', 'category'])

# index changed to ID
df.set_index('id', inplace=True)

In [4]:
df.head(10)

,text,category
id,,
611857364396965889,@aandraous @britishmuseum @AndrewsAntonio Merc...,nocode
614484565059596288,Dorian Gray with Rainbow Scarf #LoveWins (from...,happy
614746522043973632,@SelectShowcase @Tate_StIves ... Replace with ...,happy
614877582664835073,@Sofabsports thank you for following me back. ...,happy
611932373039644672,@britishmuseum @TudorHistory What a beautiful ...,happy
611570404268883969,@NationalGallery @ThePoldarkian I have always ...,happy
614456889863208960,"@britishmuseum say wot, mate?",nocode
614016385442807809,Two workshops on evaluating audience engagemen...,nocode
610916556751642624,"A Forest Road, by Thomas Gainsborough 1750 Oil...",nocode


In [5]:
df.category.value_counts() # it counts How many times each unique instance occur in your data

nocode               1572
happy                1137
not-relevant          214
angry                  57
surprise               35
sad                    32
happy|surprise         11
happy|sad               9
disgust|angry           7
disgust                 6
sad|disgust             2
sad|angry               2
sad|disgust|angry       1
Name: category, dtype: int64

In [6]:
# Removing the tweet with multiple category/nocode
df = df[~df.category.str.contains('\|')]
                    #str -> As we have to pull-it out of the string
                        #contain -> if the str contaion '|' -> Return True, Else False

In [7]:
df = df[df.category != 'nocode']

In [8]:
df.category.value_counts()

happy           1137
not-relevant     214
angry             57
surprise          35
sad               32
disgust            6
Name: category, dtype: int64

In [9]:
possible_labels = df.category.unique() # Now we have the list that conatin all-of the labels

In [10]:
label_dict = {} # Creating an empty Dict, & Looping over the possible labels
for index, possible_label in enumerate(possible_labels):
    label_dict[possible_label] = index

In [11]:
label_dict

{'happy': 0,
 'not-relevant': 1,
 'angry': 2,
 'disgust': 3,
 'sad': 4,
 'surprise': 5}

In [12]:
df['label'] = df.category.replace(label_dict)

In [13]:
df.head(10)

,text,category,label
id,,,
614484565059596288,Dorian Gray with Rainbow Scarf #LoveWins (from...,happy,0
614746522043973632,@SelectShowcase @Tate_StIves ... Replace with ...,happy,0
614877582664835073,@Sofabsports thank you for following me back. ...,happy,0
611932373039644672,@britishmuseum @TudorHistory What a beautiful ...,happy,0
611570404268883969,@NationalGallery @ThePoldarkian I have always ...,happy,0
614499696015503361,Lucky @FitzMuseum_UK! Good luck @MirandaStearn...,happy,0
613601881441570816,Yr 9 art students are off to the @britishmuseu...,happy,0
613696526297210880,@RAMMuseum Please vote for us as @sainsbury #s...,not-relevant,1
610746718641102848,#AskTheGallery Have you got plans to privatise...,not-relevant,1


# __1. Training/Validation Split__

In [14]:
from sklearn.model_selection import train_test_split

In [15]:
x_train, x_val, y_train, y_val =  train_test_split(df.index.values,
                                                   df.label.values, # Random split based on index and label
                                                   test_size=0.15, # 80/20 or 15/85 percentage
                                                   random_state=17, # Seed
                                                   stratify=df.label.values # Equal classes percentage
)

In [20]:
# New column data_type, Initally 'not_set' for all the samples
df['data_type'] = ['not_set']*df.shape[0]

In [21]:
df.head()

,text,category,label,data_type
id,,,,
614484565059596288,Dorian Gray with Rainbow Scarf #LoveWins (from...,happy,0,not_set
614746522043973632,@SelectShowcase @Tate_StIves ... Replace with ...,happy,0,not_set
614877582664835073,@Sofabsports thank you for following me back. ...,happy,0,not_set
611932373039644672,@britishmuseum @TudorHistory What a beautiful ...,happy,0,not_set
611570404268883969,@NationalGallery @ThePoldarkian I have always ...,happy,0,not_set


In [22]:
df.loc[x_train, 'data_type'] = 'train'
df.loc[x_val, 'data_type'] = 'val'

In [23]:
df.groupby(['category', 'label', 'data_type']).count()

text
category     label data_type      
angry        2     train        48
                   val           9
disgust      3     train         5
                   val           1
happy        0     train       966
                   val         171
not-relevant 1     train       182
                   val          32
sad          4     train        27
                   val           5
surprise     5     train        30
                   val           5

# __4. Loading Tokenizer and Encoding our Data__

In [24]:
! pip install transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.7/7.7 MB 54.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.0/302.0 kB 35.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.8/3.8 MB 74.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 69.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.0/295.0 kB 31.0 MB/s eta 0:00:00


In [25]:
from transformers import BertTokenizer
from torch.utils.data import TensorDataset

In [27]:
# Initialize the BERT tokenizer from a pre-trained model.
# 'bert-base-uncased' is a pre-trained model that handles lowercase text.
# Setting 'do_lower_case' to True ensures that all text is converted to lowercase during tokenization.

tokenizer = BertTokenizer.from_pretrained('bert-base-uncased',
                                          do_lower_case=True)

### __Encoding__


Convert all the Tweets into the encoded form.



In [46]:
# Encoding the Training data using the BERT tokenizer

# Use the BERT tokenizer to convert the text data in the 'train' subset to numerical representations.
encoded_data_train = tokenizer.batch_encode_plus(
    df[df.data_type=='train'].text.values,  # Select text data from the 'train' subset
    add_special_tokens=True,  # Add special tokens (e.g., [CLS], [SEP]) to the encoded sequences
    return_attention_mask=True,  # Generate attention masks to indicate token importance
    pad_to_max_length=True,  # Ensure that all sequences have the same length (padded or truncated)
    max_length=256,  # Set the maximum sequence length to 256 tokens
    return_tensors='pt'  # Return PyTorch tensors for the encoded data
)

# Encoding the Validation data using the BERT tokenizer

# Apply the BERT tokenizer to convert text data in the 'val' subset to numerical representations.
encoded_data_val = tokenizer.batch_encode_plus(
    df[df.data_type=='val'].text.values,  # Choose text data from the 'val' subset
    add_special_tokens=True,  # Include special tokens like [CLS] and [SEP]
    return_attention_mask=True,  # Create attention masks to identify important tokens
    pad_to_max_length=True,  # Make sure all sequences have the same length (padded or truncated)
    max_length=256,  # Define a maximum sequence length of 256 tokens
    return_tensors='pt'  # Return PyTorch tensors for the encoded data
)

# Spliting the data for BERT training

# When training a BERT model, we need the following components:

# Inputs ids: Encoded token IDs for each sequence.
input_ids_train = encoded_data_train['input_ids']

# Attention Masks: These masks indicate which tokens are important (1 for tokens, 0 for padding).
attention_masks_train = encoded_data_train['attention_mask']

# Labels: Corresponding labels for the training data.
labels_train = torch.tensor(df[df.data_type=='train'].label.values)

# For the validation set, we also need inputs IDs, attention masks, and labels.
# Extract input IDs, attention masks, and labels for the validation set.
input_ids_val = encoded_data_val['input_ids']
attention_masks_val = encoded_data_val['attention_mask']
labels_val = torch.tensor(df[df.data_type=='val'].label.values)


/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:2622: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  warnings.warn(


In [29]:
# Creating two different datasets for training and validation

# Create a training dataset from the training set inputs, attention masks, and labels.
dataset_train = TensorDataset(input_ids_train, attention_masks_train, labels_train)

# Create a validation dataset from the validation set inputs, attention masks, and labels.
dataset_val = TensorDataset(input_ids_val, attention_masks_val, labels_val)

In [50]:
# Keeping the 80/20
len(dataset_train)

1258

In [49]:
len(dataset_val)

223

# __5. Setting up BERT Pretrained Model__

In [54]:
# Creating a BERT-based Sequence Classification Model

# Initialize a BERT model for sequence classification. The model is loaded from the pre-trained 'bert-base-uncased'.
from transformers import BertForSequenceClassification

model = BertForSequenceClassification.from_pretrained("bert-base-uncased",
                                                      num_labels=len(label_dict),
                                                      output_attentions=False,
                                                      output_hidden_states=False)


Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


# __6. Creating Data Loaders__

In [55]:
# Importing data loader components from PyTorch to create "mini-batches"

# DataLoader: A PyTorch utility for efficiently loading data in mini-batches during training.
from torch.utils.data import DataLoader

# RandomSampler: A sampler that shuffles the data randomly to create mini-batches.
from torch.utils.data import RandomSampler

# SequentialSampler: A sampler that creates mini-batches in a sequential order, without shuffling.
from torch.utils.data import SequentialSampler


In [56]:
# Defining Data Loaders for Training and Validation

# Define the batch size, which determines the number of data samples processed in each iteration.
batch_size = 32

# Create a data loader for training data.
dataloader_train = DataLoader(dataset_train,
                              sampler=RandomSampler(dataset_train),  # Randomly shuffle data within each batch
                              batch_size=batch_size)  # Set the batch size

# Create a data loader for validation data.
dataloader_validation = DataLoader(dataset_val,
                                   sampler=RandomSampler(dataset_val),  # Randomly shuffle data within each batch
                                   batch_size=batch_size)  # Set the batch size


#  __7. Setting Up Optimiser and Scheduler__

In [57]:
# Importing Optimizer and Learning Rate Scheduler for Fine-Tuning BERT

# AdamW: An optimizer that is a variation of the Adam optimizer, specifically designed for fine-tuning transformer models like BERT.
from transformers import AdamW

# get_linear_schedule_with_warmup: A learning rate scheduler that provides linear warm-up and linear decay of the learning rate during training.
from transformers import get_linear_schedule_with_warmup


In [58]:
# Setting Up the Optimizer with Learning Rate

# Initialize an AdamW optimizer for fine-tuning the model.
# The learning rate (lr) is set to 1e-5, which falls within the recommended range from the original BERT paper.
# The epsilon (eps) value is set to 1e-8 for numerical stability.
optimizer = AdamW(model.parameters(), lr=1e-5, eps=1e-8)


/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


In [59]:
# Setting Up a Learning Rate Scheduler with Warm-up

# Define the number of training epochs.
epochs = 10

# Create a linear learning rate scheduler with warm-up for the optimizer.
# - `optimizer`: The optimizer for which the learning rate schedule is created.
# - `num_warmup_steps`: Number of warm-up steps, in this case, set to 0 for no warm-up.
# - `num_training_steps`: Total number of training steps over all epochs, calculated as the number of batches times the number of epochs.
scheduler = get_linear_schedule_with_warmup(optimizer,
                                            num_warmup_steps=0,  # No warm-up phase
                                            num_training_steps=len(dataloader_train) * epochs)


# __8. Defining our Performance Metrics__

Accuracy metric approach originally used in accuracy function in [this tutorial](https://mccormickml.com/2019/07/22/BERT-fine-tuning/#41-bertforsequenceclassification).

In [60]:
import numpy as np

In [39]:
from sklearn.metrics import f1_score

In [61]:
# Define a custom function for calculating the F1 score

# The function takes 'preds' (model predictions) and 'labels' (true labels) as inputs.
def f1_score_func(preds, labels):

    # Set up the 'preds' to axis=1 (choosing the class with the highest probability).
    # Flatten the resulting array to a single iterable list.
    preds_flat = np.argmax(preds, axis=1).flatten()

    # Flatten the 'labels' array.
    labels_flat = labels.flatten()

    # Calculate and return the F1 score using the 'weighted' average as defined by sklearn.
    return f1_score(labels_flat, preds_flat, average='weighted')


In [63]:
# Define a custom function to calculate and print the accuracy per class

# The function takes 'preds' (model predictions) and 'labels' (true labels) as inputs.
def accuracy_per_class(preds, labels):

    # Create a dictionary that maps numerical labels to their corresponding text labels.
    label_dict_inverse = {v: k for k, v in label_dict.items()}

    # Set up 'preds' to axis=1 (choosing the class with the highest probability).
    # Flatten the resulting array to a single iterable list.
    preds_flat = np.argmax(preds, axis=1).flatten()

    # Flatten the 'labels' array.
    labels_flat = labels.flatten()

    # Iterate over all the unique labels
    for label in np.unique(labels_flat):
        # Extract the model predictions and true labels for the specific class.
        y_preds = preds_flat[labels_flat == label]
        y_true = labels_flat[labels_flat == label]

        # Print the class (sentiment label) and the accuracy for that class.
        print(f'Class: {label_dict_inverse[label]}')
        print(f'Accuracy: {len(y_preds[y_preds == label])}/{len(y_true)}\n')


# __9. Create a training loop to control PyTorch finetuning of BERT using CPU or GPU acceleration__

In [65]:
# Setting a Random Seed for Reproducibility

# Define a fixed seed value to make random processes reproducible.
seed_val = 17

# Set the random seed for Python's 'random' module.
random.seed(seed_val)

# Set the random seed for NumPy's random number generator.
np.random.seed(seed_val)

# Set the random seed for PyTorch's random number generator on the CPU.
torch.manual_seed(seed_val)

# Set the random seed for PyTorch's random number generator on all available GPUs (if any).
torch.cuda.manual_seed_all(seed_val)


In [66]:
# Device Selection for Model Training

# Check if a CUDA (GPU) device is available. If available, use it; otherwise, use the CPU.
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Move the model to the selected device (GPU or CPU).
model.to(device)

# Print the selected device to indicate where the model will be trained.
print(device)


cuda


In [67]:
# Define an evaluation function for model performance on validation data

# This function takes a data loader for validation data as input.
def evaluate(dataloader_val):

    # Set the model to evaluation mode to disable gradient calculations.
    model.eval()

    # Initialize variables for total loss, predicted values, and true values.
    loss_val_total = 0
    predictions, true_vals = [], []

    # Iterate over batches in the validation data loader.
    for batch in tqdm(dataloader_val):

        # Move the batch to the selected device (GPU or CPU).
        batch = tuple(b.to(device) for b in batch)

        # Define the inputs for the model, including input IDs, attention masks, and labels.
        inputs = {'input_ids':      batch[0],
                  'attention_mask': batch[1],
                  'labels':         batch[2],
                 }

        # Disable gradient calculations to save memory during evaluation.
        with torch.no_grad():
            outputs = model(**inputs)

        # Extract the loss and logits from the model's outputs.
        loss = outputs[0]
        logits = outputs[1]
        loss_val_total += loss.item()

        # Detach the logits and labels, and convert them to NumPy arrays.
        logits = logits.detach().cpu().numpy()
        label_ids = inputs['labels'].cpu().numpy()

        # Append the predicted logits and true labels to the respective lists.
        predictions.append(logits)
        true_vals.append(label_ids)

    # Calculate the average validation loss.
    loss_val_avg = loss_val_total / len(dataloader_val)

    # Concatenate predicted values and true values into NumPy arrays.
    predictions = np.concatenate(predictions, axis=0)
    true_vals = np.concatenate(true_vals, axis=0)

    # Return the average loss, predicted values, and true values.
    return loss_val_avg, predictions, true_vals


In [70]:
# Training Loop for Fine-Tuning a BERT Model

# Iterate through the specified number of training epochs.
for epoch in tqdm(range(1, epochs+1)):

    model.train()          # Set the model in training mode

    loss_train_total = 0   # Initialize the total training loss to zero

    # Set up a progress bar to monitor the training progress.
    progress_bar = tqdm(dataloader_train, desc='Epoch {:1d}'.format(epoch), leave=False, disable=False)
    for batch in progress_bar:

        model.zero_grad()  # Clear gradients (not applicable to RNNs)

        # Unpack the batch (a tuple of 3 items) and move it to the selected device.
        batch = tuple(b.to(device) for b in batch)

        # Define the inputs for the model, including input IDs, attention masks, and labels.
        inputs = {'input_ids':      batch[0],
                  'attention_mask': batch[1],
                  'labels':         batch[2],
                 }

        # Pass the inputs to the model for forward pass and obtain outputs.
        outputs = model(**inputs)

        loss = outputs[0]  # Extract the loss from model outputs
        loss_train_total += loss.item()
        loss.backward()    # Perform backpropagation

        # Gradient Clipping: Normalize the gradients to prevent exploding gradients.
        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)

        optimizer.step()   # Update model parameters using the optimizer
        scheduler.step()   # Adjust the learning rate based on the scheduler

        # Update the progress bar with the current training loss.
        progress_bar.set_postfix({'training_loss': '{:.3f}'.format(loss.item()/len(batch))})

    # Save the model state at the end of each epoch.
    torch.save(model.state_dict(), f'finetuned_BERT_epoch_{epoch}.model')

    tqdm.write(f'\nEpoch {epoch}')

    loss_train_avg = loss_train_total / len(dataloader_train)
    tqdm.write(f'Training loss: {loss_train_avg}')

    # Evaluate the model on the validation dataset to calculate validation loss and F1 score.
    val_loss, predictions, true_vals = evaluate(dataloader_validation)
    val_f1 = f1_score_func(predictions, true_vals)
    tqdm.write(f'Validation loss: {val_loss}')
    tqdm.write(f'F1 Score (Weighted): {val_f1}')


  0%|          | 0/10 [00:00<?, ?it/s]

Epoch 1:   0%|          | 0/40 [00:00<?, ?it/s]


Epoch 1
Training loss: 1.1416399598121643


  0%|          | 0/7 [00:00<?, ?it/s]

Validation loss: 0.7918221609933036
F1 Score (Weighted): 0.6953185953656175


...

Validation loss: 0.5459873378276825
F1 Score (Weighted): 0.8360829533607166


Epoch 10:   0%|          | 0/40 [00:00<?, ?it/s]


Epoch 10
Training loss: 0.23660735208541156


  0%|          | 0/7 [00:00<?, ?it/s]

Validation loss: 0.5662973544427327
F1 Score (Weighted): 0.8307099727780941


# __10. Loading finetuned BERT model and evaluate its performance__

In [71]:
# Define a BERT-based Sequence Classification Model

# Initialize a BERT model for sequence classification. The model is loaded from the pre-trained 'bert-base-uncased'.
model = BertForSequenceClassification.from_pretrained("bert-base-uncased",
                                                      num_labels=len(label_dict),  # Specify the number of output labels
                                                      output_attentions=False,  # Disable output of attention weights
                                                      output_hidden_states=False)  # Disable output of hidden states

# Move the model to the selected device (GPU or CPU).
model.to(device)


Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12,

In [74]:
model.load_state_dict(torch.load('/content/finetuned_BERT_epoch_10.model', map_location=torch.device('cpu')))

<All keys matched successfully>

In [73]:
_, predictions, true_vals = evaluate(dataloader_validation)

  0%|          | 0/7 [00:00<?, ?it/s]

In [72]:
accuracy_per_class(predictions, true_vals)

Class: happy
Accuracy: 164/171

Class: not-relevant
Accuracy: 17/32

Class: angry
Accuracy: 8/9

Class: disgust
Accuracy: 0/1

Class: sad
Accuracy: 0/5

Class: surprise
Accuracy: 1/5

